In [114]:
import json
import requests
import os
import base64
import msal
import pandas as pd
import time

GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'
TENANT_ID = "bb3cbbb8-0b73-4ec6-b9e7-30ab4ad1ccfe"
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
CLIENT_ID = "c2f3f6c2-6b15-4990-94fb-fd7924122e4f"
scopes = ["https://graph.microsoft.com/.default"]

In [115]:
# Create a (preferably long-lived app instance) which maintains the token cache.
app = msal.PublicClientApplication(CLIENT_ID, client_credential=None, authority=AUTHORITY)

In [116]:
# Listen Parameter
list_path = "C:/Users/Jens-OlePetersen/Interessensgemeinschaft Records Management (IGRM)/Administration - Documents/2023/Finanzen/"
list_filename = "Rechnungsversand 2022 Tracking Zahlungseingänge.xlsx"
list_sheetname = "Erinnerung am 1.5.23"

In [117]:
# Einlesen der Daten 
liste = pd.read_excel(list_path + list_filename, sheet_name = list_sheetname )    #csv: , sep=";"
rm_quote = lambda x: x.replace('"', '')
liste = liste.rename(columns=rm_quote)
liste = liste.set_index("ID")
liste = liste[["Nachname", "Vorname", "Mitgliedsnummer", "Mitgliedstyp", "E-Mail", "E-Mail Rechnungsversand"]]
liste

,Nachname,Vorname,Mitgliedsnummer,Mitgliedstyp,E-Mail,E-Mail Rechnungsversand
ID,,,,,,
172,Bazak,Heike,1003,Einzel,h.bazak@mfk.ch,h.bazak@mfk.ch
175,Bühler,Rolf,1006,Einzel,rolf.buehler@ndb.admin.ch,rolf.buehler@ndb.admin.ch
181,Hasler,Raphael,1012,Einzel,news@raphaelhasler.ch,news@raphaelhasler.ch
183,Hofer,Marc,1014,Einzel,marc.hofer@archiventis.ch,marc.hofer@archiventis.ch
184,Hofmann,Roger,1015,Einzel,hofmann-reist@bluewin.ch,hofmann-reist@bluewin.ch
186,Järmann,Stephan,1017,Einzel,stephan.jaermann@swissmedic.ch,stephan.jaermann@swissmedic.ch
187,Klingler,Nina,1018,Einzel,nk@ninaklingler.ch,nk@ninaklingler.ch
191,Leuenberger,Regula,1022,Einzel,regulaleu@hotmail.com,regulaleu@hotmail.com
200,Rüfenacht,Eva,1031,Einzel,eva.ruefenacht@alptransit.ch,eva.ruefenacht@alptransit.ch


In [126]:
# Mail Parameter
# Subject
subject = "Erinnerung: IGRM Rechnung Mitgliedsbeitrag 2022"
# Body Text
#with open("mail-template.txt") as f:
#        bodytext = f.read() 
# Path to attachments
attachment_path = "C:/Users/Jens-OlePetersen/Interessensgemeinschaft Records Management (IGRM)/Administration - Documents/2022/Finanzen Belege/021 Beitragsrechnungen Mitglieder/"
# Attachment filename
def attachment_filename(mitgliedsnummer):
        return f"IGRM_Mitgliedsrechnung_{mitgliedsnummer}.pdf"
# From address
from_address = "kassier@igrm.ch"

In [ ]:
# Mail Text 
def get_message_bodytext(mitgliedsnr):
    datarow = liste.loc[liste["Mitgliedsnummer"] == mitgliedsnr].iloc[0]
    bodytext = """<p>Guten Tag {} {}</p>
    <p>Am 24. November 2022 hatten wir Ihnen die Rechnung für den IGRM Jahresbeitrag 2022 als {}-Mitglied an die Mailadresse {} zugesandt.</p>
    <p>Bis jetzt konnten wir auf unserem Vereinskonto (Stand 30.04.2023) jeodch noch keinen Zahlungseingang feststellen. Darf ich Sie bitten, den offenen Betrag zeitnah zu begleichen? Anbei eine Kopie der Rechnung. </p>
    <p>Könnten Sie dies bitte prüfen? Falls der Betrag schon bezahlt wurde, würde ich mich über eine Rückmeldung freuen. Besten Dank im Voraus.</p>
    <p>Vielen Dank f&uuml;r ihre Treue zu unserem Verein.</p>
    <p>Freundliche Gr&uuml;sse <br />Jens-Ole Petersen <br />Kassier IGRM www.igrm.ch</p>
    <p>---<br />
    Jens-Ole Petersen <br />
    Scheuerrain 5 <br />
    3007 Bern <br />
    jens-ole.petersen@igrm.ch <br />
    kassier@igrm.ch <br />
    079 667 16 88</p>
    <p>Nicht vergessen: Morgen (2.5.2023) ist der IGRM Höck - Sehen wir uns?</p>
    <p>Nicht vergessen: Am 24.05.2023 ist der Event RM in the cloud - Schon angemeldet?</p>
    """ .format(datarow["Vorname"], datarow["Nachname"], datarow["Mitgliedstyp"], datarow["E-Mail Rechnungsversand"])
    return bodytext

In [121]:
def send_single_email(message):
    result = None
    accounts = app.get_accounts()
    if accounts:
        result = app.acquire_token_silent(scopes, account=accounts[0])

    if not result:
        print("No suitable token exists in cache. Let's authenticate against Azure Active Directory.")
        result = app.acquire_token_interactive(scopes=scopes)

    if "access_token" in result:
        endpoint = f'https://graph.microsoft.com/v1.0/me/sendMail'
        email_msg = {'Message': message,
                    'SaveToSentItems': 'true'}
        r = requests.post(endpoint,
                          headers={'Authorization': 'Bearer ' + result['access_token'],
                                   "Content-Type": "application/json; charset=utf-8"}, 
                          json=email_msg)
        if r.ok:
            return True
        else:
            print(r.json())
            return False
    else:
        print("Das hat nicht geklappt. :-(")
        print(result.get("error"))
        print(result.get("error_description"))
        print(result.get("correlation_id"))
        return False

In [122]:
def get_attachment_object(mitgliedsnummer):

    file_full_path = "".join([attachment_path, attachment_filename(mitgliedsnummer)]) 

    if not os.path.exists(file_full_path):
        print('file is not found')
        return
    
    with open(file_full_path, 'rb') as upload:
        media_content = base64.b64encode(upload.read())
        
    attachment_object = {
        '@odata.type': '#microsoft.graph.fileAttachment',
        'contentType': "text/plain",
        'name': attachment_filename(mitgliedsnummer),
        'contentBytes': media_content.decode('utf-8')
    }
    return attachment_object

In [123]:
def get_message_object(mitgliedsnummer, to_email_adress):

    attachment = get_attachment_object(mitgliedsnummer)
    message_object = {  'Subject': subject,
                        'Body': {'ContentType': 'HTML', 
                                 'Content': get_message_bodytext(mitgliedsnummer)},
                        'ToRecipients': [{'EmailAddress': {'Address': to_email_adress}}],
                        "from": {"emailAddress": {"address": from_address}},
                        'attachments': [ attachment ]
                        }
    return message_object

In [128]:
# Einzel Test
message = get_message_object(1003, "jens-ole@petersen.top")
r = send_single_email(message)
if not r:
    print("STOP FEHLER")

In [132]:
for i in range(len(liste)):
#for i in range(3):

    row = liste.iloc[i]

    mitgliedsnr = row["Mitgliedsnummer"] 
    zieladresse = row["E-Mail"]
    #zieladresse = "jens-ole@petersen.top"
    print("Jetzt zu senden:", row["Nachname"], row["Vorname"], mitgliedsnr, zieladresse, row["E-Mail Rechnungsversand"])

    message = get_message_object(mitgliedsnr, zieladresse)
    result = send_single_email(message)
    if result:
        print("OK.")
    else:
        print("STOP FEHLER")
    
    time.sleep(3)  # Send Limit in O365 of 30 Mail per minute

Jetzt zu senden: Bazak Heike 1003 h.bazak@mfk.ch h.bazak@mfk.ch
OK.
Jetzt zu senden: Bühler Rolf 1006 rolf.buehler@ndb.admin.ch rolf.buehler@ndb.admin.ch
OK.
Jetzt zu senden: Hasler Raphael 1012 news@raphaelhasler.ch news@raphaelhasler.ch
OK.
Jetzt zu senden: Hofer Marc 1014 marc.hofer@archiventis.ch marc.hofer@archiventis.ch
OK.
Jetzt zu senden: Hofmann Roger 1015 hofmann-reist@bluewin.ch hofmann-reist@bluewin.ch
OK.
Jetzt zu senden: Järmann Stephan 1017 stephan.jaermann@swissmedic.ch stephan.jaermann@swissmedic.ch
OK.
Jetzt zu senden: Klingler Nina 1018 nk@ninaklingler.ch nk@ninaklingler.ch
OK.
Jetzt zu senden: Leuenberger Regula 1022 regulaleu@hotmail.com regulaleu@hotmail.com
OK.
Jetzt zu senden: Rüfenacht Eva 1031 eva.ruefenacht@alptransit.ch eva.ruefenacht@alptransit.ch
OK.
Jetzt zu senden: Scheidegger-Zbinden Doris 1033 doris.scheidegger-zbinden@gs-edi.admin.ch doris.scheidegger-zbinden@gs-edi.admin.ch
OK.
Jetzt zu senden: Schwab Michael 1034 schwamic1@gmail.com schwamic1@gmail.